# Selected Social Characteristics: Educational Attainment from the American Community Survey

**[Work in progress]**

This notebook downloads [selected social characteristics (DP02)](https://data.census.gov/cedsci/table?q=DP02&tid=ACSDP5Y2018.DP02) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data 2018](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [19]:
import os
import pandas as pd
from pathlib import Path
import time

In [20]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [21]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/com.Neo4j.Relate/data/dbmss/dbms-8bf637fc-0d20-4d9f-9c6f-f7e72e92a4da/import


## Download selected variables

* [Selected social characteristics for US](https://data.census.gov/cedsci/table?q=DP02&tid=ACSDP5Y2018.DP02)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP02.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP02/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP02 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [22]:
variables = {# EDUCATIONAL ATTAINMENT
             'DP02_0058E': 'population25YearsAndOver',
             'DP02_0059E': 'lessThan9thGrade',
             'DP02_0059PE': 'lessThan9thGradePct',
             'DP02_0060E': 'grade9thTo12thNoDiploma',
             'DP02_0060PE': 'grade9thTo12thNoDiplomaPct',
             'DP02_0061E': 'highSchoolGraduate',
             'DP02_0061PE': 'highSchoolGraduatePct',
             'DP02_0062E': 'someCollegeNoDegree',
             'DP02_0062PE': 'someCollegeNoDegreePct',
             'DP02_0063E': 'associatesDegree',
             'DP02_0063PE': 'associatesDegreePct',
             'DP02_0064E': 'bachelorsDegree',
             'DP02_0064PE': 'bachelorsDegreePct',
             'DP02_0065E': 'graduateOrProfessionalDegree',
             'DP02_0065PE': 'graduateOrProfessionalDegreePct',
             'DP02_0066E': 'highSchoolGraduateOrHigher',
             'DP02_0066PE': 'highSchoolGraduateOrHigherPct',
             'DP02_0067E': 'bachelorsDegreeOrHigher',
             'DP02_0067PE': 'bachelorsDegreeOrHigherPct',
            }

In [23]:
fields = ",".join(variables.keys())

In [24]:
for v in variables.values():
    if 'Pct' in v:
        print('e.' + v + ' = toFloat(row.' + v + '),')
    else:
        print('e.' + v + ' = toInteger(row.' + v + '),')

e.population25YearsAndOver = toInteger(row.population25YearsAndOver),
e.lessThan9thGrade = toInteger(row.lessThan9thGrade),
e.lessThan9thGradePct = toFloat(row.lessThan9thGradePct),
e.grade9thTo12thNoDiploma = toInteger(row.grade9thTo12thNoDiploma),
e.grade9thTo12thNoDiplomaPct = toFloat(row.grade9thTo12thNoDiplomaPct),
e.highSchoolGraduate = toInteger(row.highSchoolGraduate),
e.highSchoolGraduatePct = toFloat(row.highSchoolGraduatePct),
e.someCollegeNoDegree = toInteger(row.someCollegeNoDegree),
e.someCollegeNoDegreePct = toFloat(row.someCollegeNoDegreePct),
e.associatesDegree = toInteger(row.associatesDegree),
e.associatesDegreePct = toFloat(row.associatesDegreePct),
e.bachelorsDegree = toInteger(row.bachelorsDegree),
e.bachelorsDegreePct = toFloat(row.bachelorsDegreePct),
e.graduateOrProfessionalDegree = toInteger(row.graduateOrProfessionalDegree),
e.graduateOrProfessionalDegreePct = toFloat(row.graduateOrProfessionalDegreePct),
e.highSchoolGraduateOrHigher = toInteger(row.highSchoo

## Download county-level data using US Census API

In [25]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [26]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,DP02_0058E,DP02_0059E,DP02_0059PE,DP02_0060E,DP02_0060PE,DP02_0061E,DP02_0061PE,DP02_0062E,DP02_0062PE,DP02_0063E,DP02_0063PE,DP02_0064E,DP02_0064PE,DP02_0065E,DP02_0065PE,DP02_0066E,DP02_0066PE,DP02_0067E,DP02_0067PE,state,county
1,2135,70,3.3,149,7.0,680,31.9,472,22.1,175,8.2,469,22.0,120,5.6,1916,89.7,589,27.6,02,195
2,3947,297,7.5,583,14.8,2117,53.6,654,16.6,106,2.7,132,3.3,58,1.5,3067,77.7,190,4.8,02,158
3,2811491,165555,5.9,188958,6.7,634810,22.6,685299,24.4,237799,8.5,571408,20.3,327662,11.7,2456978,87.4,899070,32.0,04,013
4,23531,1054,4.5,2325,9.9,7028,29.9,7105,30.2,2421,10.3,2272,9.7,1326,5.6,20152,85.6,3598,15.3,04,009


##### Add column names

In [27]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [28]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [29]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [30]:
df.head()

,population25YearsAndOver,lessThan9thGrade,lessThan9thGradePct,grade9thTo12thNoDiploma,grade9thTo12thNoDiplomaPct,highSchoolGraduate,highSchoolGraduatePct,someCollegeNoDegree,someCollegeNoDegreePct,associatesDegree,associatesDegreePct,bachelorsDegree,bachelorsDegreePct,graduateOrProfessionalDegree,graduateOrProfessionalDegreePct,highSchoolGraduateOrHigher,highSchoolGraduateOrHigherPct,bachelorsDegreeOrHigher,bachelorsDegreeOrHigherPct,stateFips,countyFips
1,2135,70,3.3,149,7.0,680,31.9,472,22.1,175,8.2,469,22.0,120,5.6,1916,89.7,589,27.6,02,195
2,3947,297,7.5,583,14.8,2117,53.6,654,16.6,106,2.7,132,3.3,58,1.5,3067,77.7,190,4.8,02,158
3,2811491,165555,5.9,188958,6.7,634810,22.6,685299,24.4,237799,8.5,571408,20.3,327662,11.7,2456978,87.4,899070,32.0,04,013
4,23531,1054,4.5,2325,9.9,7028,29.9,7105,30.2,2421,10.3,2272,9.7,1326,5.6,20152,85.6,3598,15.3,04,009
5,29228,3404,11.6,3723,12.7,7957,27.2,6308,21.6,1698,5.8,3993,13.7,2145,7.3,22101,75.6,6138,21.0,04,023


In [31]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,population25YearsAndOver,lessThan9thGrade,lessThan9thGradePct,grade9thTo12thNoDiploma,grade9thTo12thNoDiplomaPct,highSchoolGraduate,highSchoolGraduatePct,someCollegeNoDegree,someCollegeNoDegreePct,associatesDegree,associatesDegreePct,bachelorsDegree,bachelorsDegreePct,graduateOrProfessionalDegree,graduateOrProfessionalDegreePct,highSchoolGraduateOrHigher,highSchoolGraduateOrHigherPct,bachelorsDegreeOrHigher,bachelorsDegreeOrHigherPct,stateFips,countyFips
897,2223376,149313,6.7,136695,6.1,409272,18.4,498566,22.4,181508,8.2,521525,23.5,326497,14.7,1937368,87.1,848022,38.1,06,073


In [32]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [33]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP02EducationAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [34]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [35]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,DP02_0058E,DP02_0059E,DP02_0059PE,DP02_0060E,DP02_0060PE,DP02_0061E,DP02_0061PE,DP02_0062E,DP02_0062PE,DP02_0063E,DP02_0063PE,DP02_0064E,DP02_0064PE,DP02_0065E,DP02_0065PE,DP02_0066E,DP02_0066PE,DP02_0067E,DP02_0067PE,state,zip code tabulation area
1,1709,94,5.5,171,10.0,620,36.3,379,22.2,186,10.9,131,7.7,128,7.5,1444,84.5,259,15.2,51,23833
2,863,43,5.0,186,21.6,327,37.9,137,15.9,107,12.4,63,7.3,0,0.0,634,73.5,63,7.3,51,23850
3,9191,471,5.1,1132,12.3,2688,29.2,2199,23.9,914,9.9,1148,12.5,639,7.0,7588,82.6,1787,19.4,51,23851
4,2654,205,7.7,243,9.2,1047,39.4,517,19.5,192,7.2,274,10.3,176,6.6,2206,83.1,450,17.0,51,23890


##### Add column names

In [38]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('stateFips')
columns.append('postalCode')
df.columns = columns

In [39]:
df.head()

,population25YearsAndOver,lessThan9thGrade,lessThan9thGradePct,grade9thTo12thNoDiploma,grade9thTo12thNoDiplomaPct,highSchoolGraduate,highSchoolGraduatePct,someCollegeNoDegree,someCollegeNoDegreePct,associatesDegree,associatesDegreePct,bachelorsDegree,bachelorsDegreePct,graduateOrProfessionalDegree,graduateOrProfessionalDegreePct,highSchoolGraduateOrHigher,highSchoolGraduateOrHigherPct,bachelorsDegreeOrHigher,bachelorsDegreeOrHigherPct,stateFips,postalCode
2,863,43,5.0,186,21.6,327,37.9,137,15.9,107,12.4,63,7.3,0,0.0,634,73.5,63,7.3,51,23850
3,9191,471,5.1,1132,12.3,2688,29.2,2199,23.9,914,9.9,1148,12.5,639,7.0,7588,82.6,1787,19.4,51,23851
4,2654,205,7.7,243,9.2,1047,39.4,517,19.5,192,7.2,274,10.3,176,6.6,2206,83.1,450,17.0,51,23890
5,164,49,29.9,0,0.0,17,10.4,11,6.7,46,28.0,28,17.1,13,7.9,115,70.1,41,25.0,51,23302
6,406,10,2.5,82,20.2,194,47.8,104,25.6,16,3.9,0,0.0,0,0.0,314,77.3,0,0.0,51,23307


In [40]:
# Example data
df.query("postalCode == '90210'")

,population25YearsAndOver,lessThan9thGrade,lessThan9thGradePct,grade9thTo12thNoDiploma,grade9thTo12thNoDiplomaPct,highSchoolGraduate,highSchoolGraduatePct,someCollegeNoDegree,someCollegeNoDegreePct,associatesDegree,associatesDegreePct,bachelorsDegree,bachelorsDegreePct,graduateOrProfessionalDegree,graduateOrProfessionalDegreePct,highSchoolGraduateOrHigher,highSchoolGraduateOrHigherPct,bachelorsDegreeOrHigher,bachelorsDegreeOrHigherPct,stateFips,postalCode
24526,15118,390,2.6,414,2.7,1720,11.4,2019,13.4,791,5.2,4935,32.6,4849,32.1,14314,94.7,9784,64.7,06,90210


In [41]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [42]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP02EducationZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [43]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [44]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [45]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [46]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [47]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,population25YearsAndOver,lessThan9thGrade,lessThan9thGradePct,grade9thTo12thNoDiploma,grade9thTo12thNoDiplomaPct,highSchoolGraduate,highSchoolGraduatePct,someCollegeNoDegree,someCollegeNoDegreePct,associatesDegree,associatesDegreePct,bachelorsDegree,bachelorsDegreePct,graduateOrProfessionalDegree,graduateOrProfessionalDegreePct,highSchoolGraduateOrHigher,highSchoolGraduateOrHigherPct,bachelorsDegreeOrHigher,bachelorsDegreeOrHigherPct,stateFips,countyFips,tract,source,aggregationLevel
6,1000,0,0.0,30,3.0,74,7.4,114,11.4,31,3.1,418,41.8,333,33.3,970,97.0,751,75.1,06,073,06073008339,American Community Survey 5 year,Tract
7,5183,340,6.6,156,3.0,901,17.4,1016,19.6,443,8.5,1305,25.2,1022,19.7,4687,90.4,2327,44.9,06,073,06073008347,American Community Survey 5 year,Tract
8,6942,248,3.6,269,3.9,368,5.3,1199,17.3,730,10.5,2759,39.7,1369,19.7,6425,92.6,4128,59.5,06,073,06073008354,American Community Survey 5 year,Tract
9,4799,347,7.2,270,5.6,1038,21.6,1065,22.2,328,6.8,1080,22.5,671,14.0,4182,87.1,1751,36.5,06,073,06073008505,American Community Survey 5 year,Tract
10,5187,201,3.9,163,3.1,525,10.1,1063,20.5,452,8.7,1682,32.4,1101,21.2,4823,93.0,2783,53.7,06,073,06073017604,American Community Survey 5 year,Tract


### Save data

In [48]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP02EducationTract.csv", index=False)

In [49]:
df.shape

(73056, 24)